In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, Flatten, AvgPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [3]:
SEED = np.random.seed(69)
PATH = '/content/drive/My Drive/Colab Notebooks/AiDesignerBot/'

In [4]:
def load_data():
    labels = pd.read_csv(PATH + 'other_clf_sample.csv')

    train_datagen = ImageDataGenerator(validation_split=.25, rescale=1/255., horizontal_flip=True, vertical_flip=True, rotation_range=45)
    train_data = train_datagen.flow_from_dataframe(dataframe=labels, directory=PATH + 'images/',
                                                   x_col='title', y_col='target', target_size=(225, 225),
                                                   batch_size=32, class_mode='raw', subset='training', seed=SEED)
    
    test_datagen = ImageDataGenerator(validation_split=.25, rescale=1/255.)
    test_data = test_datagen.flow_from_dataframe(dataframe=labels, directory=PATH + 'images/',
                                                 x_col='title', y_col='target', target_size=(225, 225),
                                                 batch_size=32, class_mode='raw', subset='validation', seed=SEED)
    
    return train_data, test_data

In [5]:
def create_model(input_shape):
    model = Sequential()
    optimizer = Adam(lr=1e-3)
    model.add(Conv2D(filters=6, kernel_size=(5, 5), padding='same', input_shape=input_shape, activation='relu'))
    model.add(AvgPool2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=12, kernel_size=(5, 5), padding='valid', activation='relu'))
    model.add(AvgPool2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
    return model

In [6]:
def train_model(model, train_data, test_data, batch_size=None, epochs=10,
               steps_per_epoch=None, validation_steps=None):
    if steps_per_epoch == None:
        steps_per_epoch = len(train_data)
    if validation_steps == None:
        validation_steps = len(test_data)

    model.fit(train_data, 
              validation_data=test_data,
              batch_size=batch_size, 
              epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              verbose=1, 
              shuffle=True,
              validation_steps=validation_steps)

    return model

In [7]:
model = create_model((225, 225, 3),)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 225, 225, 6)       456       
_________________________________________________________________
average_pooling2d (AveragePo (None, 112, 112, 6)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 108, 108, 12)      1812      
_________________________________________________________________
average_pooling2d_1 (Average (None, 54, 54, 12)        0         
_________________________________________________________________
flatten (Flatten)            (None, 34992)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                1119776   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3

In [8]:
train, test = load_data()

Found 696 validated image filenames.
Found 232 validated image filenames.


In [9]:
pretrained_model = train_model(model, train, test, epochs=10)

Epoch 1/10
22/22 [==============================] - 30s 1s/step - loss: 0.9808 - acc: 0.6977 - val_loss: 0.5424 - val_acc: 0.7888
Epoch 2/10
22/22 [==============================] - 25s 1s/step - loss: 0.5379 - acc: 0.7875 - val_loss: 0.4729 - val_acc: 0.8017
Epoch 3/10
22/22 [==============================] - 26s 1s/step - loss: 0.4860 - acc: 0.8012 - val_loss: 0.4604 - val_acc: 0.8750
Epoch 4/10
22/22 [==============================] - 26s 1s/step - loss: 0.5065 - acc: 0.7684 - val_loss: 0.4970 - val_acc: 0.8233
Epoch 5/10
22/22 [==============================] - 26s 1s/step - loss: 0.4950 - acc: 0.7842 - val_loss: 0.4704 - val_acc: 0.8836
Epoch 6/10
22/22 [==============================] - 26s 1s/step - loss: 0.4450 - acc: 0.8080 - val_loss: 0.4293 - val_acc: 0.8103
Epoch 7/10
22/22 [==============================] - 26s 1s/step - loss: 0.4788 - acc: 0.7988 - val_loss: 0.4182 - val_acc: 0.8836
Epoch 8/10
22/22 [==============================] - 26s 1s/step - loss: 0.4311 - acc: 0.84

In [12]:
pretrained_model.save('other_clf_model')

INFO:tensorflow:Assets written to: other_clf_model/assets


In [13]:
!cp -r other_clf_model '/content/drive/My Drive/Colab Notebooks/AiDesignerBot/'